In [1]:
# librairies

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# open the combined_square_storm_dataset.nc file

sq = xr.open_dataset('combined_square_storm_dataset.nc')
sq

<xarray.Dataset>
Dimensions:     (point: 158, coordinate: 4, storm: 96)
Coordinates:
  * point       (point) int32 0 1 2 3 4 5 6 7 ... 151 152 153 154 155 156 157
  * coordinate  (coordinate) int32 0 1 2 3
Dimensions without coordinates: storm
Data variables:
    tc_irad     (storm, point, coordinate) float64 ...

In [3]:
# path to one variable 

local_path = '/Users/fabienaugsburger/myCloud/Unil/Master/Thesis/Datasets/Curnagl/ERA5_1990_2m_dewpoint_temperature.nc'
dew_point = xr.open_dataset(local_path)